Import neccesary libraries for **functionality**

In [ ]:
import tkinter as tk
import ast as ast
import matplotlib.pyplot as plt
import numpy as np

# LibCST basic Functionality Tests
General applications of LibCST (https://github.com/Instagram/LibCST)

Setup functions, all required to be run in sequence for tests below

In [ ]:
# install libCST
!pip install libcst

In [ ]:
# libCST functionality
import libcst
from libcst.tool import dump
from libcst.metadata import MetadataWrapper, ParentNodeProvider

In [ ]:
# note that only one python file input is assumed, otherwise it only saves the data of the last file
try:
    from google.colab import files
    uploaded = files.upload()
except ImportError as e:
    pass


filenames = uploaded.keys()

for file in filenames:
    data = uploaded[file]

Assorted LibCST tests (requires above file upload to work)

Dump the entire CST for the imported data

In [ ]:
# convert given file to libCST (https://libcst.readthedocs.io/en/latest/parser.html)
print(dump(libcst.parse_module(data)))

Print scrope of every function in the CST

In [ ]:
# or convert and see scope (https://libcst.readthedocs.io/en/latest/scope_tutorial.html)
wrapper = libcst.metadata.MetadataWrapper(libcst.parse_module(data))
scopes = set(wrapper.resolve(libcst.metadata.ScopeProvider).values())
for scope in scopes:
    print(scope)

Test function given by LibCST docs, prints all simple strings in a function if the function name is not 'foo'

In [ ]:
# if function name is not foo, print any simple strings found inside (https://libcst.readthedocs.io/en/latest/visitors.html#visit-and-leave-helper-functions)
class FooingAround(libcst.CSTVisitor):
    def visit_FunctionDef(self, node: libcst.FunctionDef) -> bool:
        print(node.name.value)

    def visit_SimpleString(self, node: libcst.SimpleString) -> None:
        print(node.value)


#demo = libcst.parse_module("'abc'\n'123'\ndef foo():\n    'not printed'\ndef blah():\n    'printed'")
demo = libcst.parse_module(data)

_ = demo.visit(FooingAround())

Print function names found in CST or function calls

In [ ]:
# print function names
class FuncNames(libcst.CSTVisitor):
    def visit_FunctionDef(self, node: libcst.FunctionDef) -> bool:
        print(node.name.value)

func_names = libcst.parse_module(data)

_ = func_names.visit(FuncNames())

In [ ]:
# print function calls
class FuncCalls(libcst.CSTVisitor):
    # (https://libcst.readthedocs.io/_/downloads/en/latest/pdf/#page=53&zoom=auto,-205,721)
    def visit_Call(self, node: libcst.Call) -> bool:
      if (type(node.func)) is libcst._nodes.expression.Name:
        print("Function call name: "+node.func.value)
      elif (type(node.func)) is libcst._nodes.expression.Attribute:
        print("Function call attribute: "+node.func.value.value+"."+node.func.attr.value)

func_calls = libcst.parse_module(data)

_ = func_calls.visit(FuncCalls())

Print function names and import aliases found in CST

In [ ]:
# print function names and import aliases
class FuncNames(libcst.CSTVisitor):
    def visit_FunctionDef(self, node: libcst.FunctionDef) -> bool:
        print(node.name.value+" is a function def")

    def visit_ImportAlias(self, node: libcst.ImportAlias) -> bool:
        curr=node.asname
        # the import may not specify an alias, only print aliases of imported functions
        # https://libcst.readthedocs.io/_/downloads/en/latest/pdf/#page=78&zoom=auto,-205,314
        if curr:
          print(curr.name.value+ " is an alias of imported "+node.name.value)


#demo = libcst.parse_module("'abc'\n'123'\ndef foo():\n    'not printed'\ndef blah():\n    'printed'")
all_func_names = libcst.parse_module(data)

_ = all_func_names.visit(FuncNames())

Function renamer (simple implementation, incomplete, mainly to test transformers)

In [ ]:
# edit all function names if encountered (note that changes with transforms happen on leaving a node, so name functions leave_NodeType)

# for choosing a function name
import random

# placeholder list to replace function names with
colors = ['aliceblue','antiquewhite','aqua','aquamarine','azure','beige','bisque','black','blanchedalmond',
          'blue','blueviolet','brown','burlywood','cadetblue','chartreuse','chocolate','coral','cornflowerblue',
          'cornsilk','crimson','cyan','darkblue','darkcyan','darkgoldenrod','darkgray','darkgreen','darkkhaki',
          'darkmagenta','darkolivegreen','darkorange','darkorchid','darkred','darksalmon','darkseagreen',
          'darkslateblue','darkslategray','darkturquoise','darkviolet','deeppink','deepskyblue','dimgray',
          'dodgerblue','firebrick','floralwhite','forestgreen','fuchsia','gainsboro','ghostwhite','gold','goldenrod',
          'gray','green','greenyellow','honeydew','hotpink','indianred','indigo','ivory','khaki','lavender',
          'lavenderblush','lawngreen','lemonchiffon','lightblue','lightcoral','lightcyan','lightgoldenrodyellow',
          'lightgreen','lightgray','lightpink','lightsalmon','lightseagreen','lightskyblue','lightslategray',
          'lightsteelblue','lightyellow','lime','limegreen','linen','magenta','maroon','mediumaquamarine','mediumblue',
          'mediumorchid','mediumpurple','mediumseagreen','mediumslateblue','mediumspringgreen','mediumturquoise',
          'mediumvioletred','midnightblue','mintcream','mistyrose','moccasin','navajowhite','navy','oldlace','olive',
          'olivedrab','orange','orangered','orchid','palegoldenrod','palegreen','paleturquoise','palevioletred',
          'papayawhip','peachpuff','peru','pink','plum','powderblue','purple','red','rosybrown','royalblue','saddlebrown',
          'salmon','sandybrown','seagreen','seashell','sienna','silver','skyblue','slateblue','slategray','snow',
          'springgreen','steelblue','tan','teal','thistle','tomato','turquoise','violet','wheat','white','whitesmoke'
          ,'yellow','yellowgreen']


#dict of existing pairs, will be used to replace all calls of old function with calls to new name
# assuming that multiple funcs don't have the same new name
func_name_pairs = dict()

# rename all function defs or aliases (only rename custom functions, not things like print() or math.log)
class FuncRename(libcst.CSTTransformer):

  # rename function names in a "def funcname: " node
  # FunctionDef node docs: (https://libcst.readthedocs.io/_/downloads/en/latest/pdf/#page=73&zoom=auto,-205,215)
  def leave_FunctionDef(self, node: libcst.FunctionDef, updated_node: libcst.FunctionDef) -> libcst.FunctionDef:
    if updated_node.name.value not in func_name_pairs:
      new_name=random.choice(colors)
      colors[colors.index(new_name)]=str(new_name+str(1))
      func_name_pairs.update({updated_node.name.value: new_name})
    # the name node in function def is a child node, thus to change function name via the FunctionDef parent node, use with_deep_changes via:
    # (https://libcst.readthedocs.io/en/latest/nodes.html#libcst.CSTNode.with_deep_changes)

    #print("Function def of \'"+updated_node.name.value+"\' has been renamed to \'"+func_name_pairs[updated_node.name.value]+"\'")
    return updated_node.with_deep_changes(updated_node.name, value=func_name_pairs[updated_node.name.value])

  # rename function names in a "import x as y" node
  # ImportAlias node docs: (https://libcst.readthedocs.io/_/downloads/en/latest/pdf/#page=78&zoom=auto,-205,314)
  def leave_ImportAlias(self, node: libcst.ImportAlias, updated_node: libcst.ImportAlias) -> libcst.ImportAlias:
    alias_node=updated_node.asname
    if alias_node:
      if alias_node.name.value not in func_name_pairs:
        new_name=random.choice(colors)
        colors[colors.index(new_name)]=str(new_name+str(1))
        func_name_pairs.update({alias_node.name.value: new_name})

      #print("Import alias of \'"+alias_node.name.value+"\' has been renamed to \'"+func_name_pairs[alias_node.name.value]+"\'")
      return updated_node.with_deep_changes(updated_node.asname.name, value=func_name_pairs[alias_node.name.value])
    return updated_node


# kept separate from FuncRename to do two-pass and prevent renaming predefined functions like print()
class CallRename(libcst.CSTTransformer):

  # rename function names in a function call node
  # Call node docs: (https://libcst.readthedocs.io/_/downloads/en/latest/pdf/#page=53&zoom=auto,-205,721)
  def leave_Call(self, node: libcst.Call, updated_node: libcst.Call) -> libcst.Call:

    # Name node: (https://libcst.readthedocs.io/_/downloads/en/latest/pdf/#page=48&zoom=auto,-205,344)
    if (type(updated_node.func)) is libcst._nodes.expression.Name:
      if (updated_node.func.value) in func_name_pairs:

        #print("Function call of \'"+updated_node.func.value+"\' has been renamed to \'"+func_name_pairs[updated_node.func.value]+"\'")
        return updated_node.with_deep_changes(updated_node.func, value=func_name_pairs[updated_node.func.value])

    # for attributes, aka package.function(), only the package can be potentially custom (e.g. 'import math as blah', 'blah.log()')
    # as if you import a subsect and rename it, it will be a normal function call (e.g. 'from math import log as blah', funct call would be
    #                                                                                   'blah()' not 'math.blah()'
    # Attribute node: (https://libcst.readthedocs.io/_/downloads/en/latest/pdf/#page=48&zoom=auto,-205,344)
    elif (type(updated_node.func)) is libcst._nodes.expression.Attribute:
      if updated_node.func.value.value in func_name_pairs:

        #print("Function call of \'"+updated_node.func.value.value+"."+updated_node.func.attr.value+"\' has been renamed to \'"+func_name_pairs[updated_node.func.value.value]+"."+updated_node.func.attr.value+"\'")
        return updated_node.with_deep_changes(updated_node.func.value, value=func_name_pairs[updated_node.func.value.value])
    return updated_node

orig = libcst.parse_module(data)
funcs_renamed = orig.visit(FuncRename())
calls_renamed = funcs_renamed.visit(CallRename())

#print(func_name_pairs)


How to access code with renamed functions

In [ ]:
# print converted code
print(calls_renamed.code)

In [ ]:
# or auto-download
with open('renamed_funcs.py', 'w+') as file:
    file.write(calls_renamed.code)
file.close

from google.colab import files
files.download("renamed_funcs.py")

# 'ast' Python Syntax Analyzer
General applications of "ast" library (https://dev.to/marvinjude/abstract-syntax-trees-and-practical-applications-in-javascript-4a3)

In [ ]:
file_path = "md5 hash.py"

try:
    with open(file_path, "r") as python_file:
        print(f"Opened {file_path} successfully")

        code_string = python_file.read()

        try:
            ast.parse(code_string)
            print(f"Python file: '{file_path}' is syntactically valid")

        except SyntaxError as error:
            print(f"Syntax error in file '{file_path}': {error}")

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")

except IOError:
    print(f"Error: An error occurred while opening the file '{file_path}'")



# LibCST basic Functionality Tests (Bryan's Branch)
General applications of LibCST (https://github.com/Instagram/LibCST)

In [ ]:
import libcst as cst
from libcst.metadata import MetadataWrapper, ParentNodeProvider
import google.generativeai as genai

model = genai.GenerativeModel("gemini-1.5-flash")

genai.configure(api_key="custom-key")

filename = "/content/sample_data/md5 hash.py"
with open(filename, "r") as file:
    source_code = file.read()

module = cst.parse_module(source_code)
wrapped_module = MetadataWrapper(module)

class VarRename(cst.CSTTransformer):
    METADATA_DEPENDENCIES = (ParentNodeProvider,)

    def __init__(self):
        self.variable_map = {}

    def get_synonym(self, original_varname):

        if original_varname not in self.variable_map:

            try:

                synonym = model.generate_content(f"Provide a one word synonym for '{original_varname}' in a coding context. This is for a school project and requires code variables to be swapped with one word.")
                synonym = synonym.candidates[0].content.parts[0].text.strip()
                print(synonym)
                self.variable_map[original_varname] = synonym.text  # Cache the synonym

            except Exception as e:
                print(f"Error fetching synonym for '{original_varname}':", e)
                self.variable_map[original_varname] = original_varname  # Fallback to original name if there's an error

        return self.variable_map[original_varname]

    def leave_Name(self, original_node, updated_node):
        # Access the parent node using metadata
        parent = self.get_metadata(ParentNodeProvider, original_node)

        # Check if the parent is an AssignTarget or Param (contexts where variables are defined)
        if isinstance(parent, (cst.AssignTarget, cst.Param)):
            new_name = self.get_synonym(updated_node.value)
            return updated_node.with_changes(value=new_name)

        return updated_node

# Apply the transformation
transformed_cst = wrapped_module.visit(VarRename())
modified_code = transformed_cst.code

# Write the modified code to a new file
output_filename = "your_script_renamed.py"
with open(output_filename, "w") as output_file:
    output_file.write(modified_code)

print(f"Transformed code has been saved to {output_filename}")
